In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pyautogui
import time
import re
import csv
import string
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import nbformat as nbf
import json
import yaml


In [2]:
def opening_url(page_link):
    
    driver_path = 'D:\Projetos_git\chromedriver'
    
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(driver_path) #, options=chrome_options)

    driver.get(page_link)
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    
    return driver

In [3]:
def credentials(driver):
    
    conf = yaml.load(open('credentials.yml'))
    
#    credentials.yml:

#    text file with a structure like below:
    
    email = conf['user']['email']
    pwd = conf['user']['password']
    driver.find_element_by_xpath("//input[@name='uname']").send_keys(email)
    driver.find_element_by_xpath("//input[@name='pw']").send_keys(pwd)
    driver.find_element_by_xpath("//input[@name='pw']").send_keys(Keys.ENTER)
    time.sleep(3)

    driver.refresh()
    time.sleep(3)

In [4]:
def topics_links(driver):
    
    elements = driver.find_elements_by_xpath("//a[contains(@href, '/python/')]")

    tasks = []
    links = []

    for elem in elements:
        links.append(elem.get_attribute('href'))
        el = elem.find_elements_by_xpath(".//span[contains(@class, 'h2')]")
        for e in el:
            tasks.append(e.text)

    return tasks, links

In [5]:
def exercises_links(item):
    
    driver.get(item)
    time.sleep(3)
    elements = driver.find_elements_by_xpath("//a[contains(@href,'/prob')]")

    prob = []

    for elem in elements:
        prob.append(elem.get_attribute("href"))
    
    return prob

In [6]:
def zoom_out():
    
    time.sleep(2)
    pyautogui.moveTo(1300,500)
    pyautogui.keyDown("ctrl")
    pyautogui.scroll(-300)
    time.sleep(.5)
    pyautogui.scroll(-300)
    time.sleep(.5)
    pyautogui.scroll(-300)
    time.sleep(.5)
    pyautogui.scroll(-300)
    pyautogui.keyUp("ctrl")
    pyautogui.scroll(-90)
    pyautogui.click()

def printscreen(title):
    img = pyautogui.screenshot(region=(5,145, 1220, 900))
    title_name = title + ".png"
    img.save(title_name)
    
    return title_name

In [19]:
def get_content(item):
    
    driver.get(item)
    time.sleep(4)
    driver.find_element_by_xpath("//button[contains(@class,'go')]").send_keys(Keys.ENTER)

    time.sleep(3)

    elements = driver.find_elements_by_xpath("//*")

    text = []

    for elem in elements:
        text.append(elem.text)

    text = text[0]
    
    return text

In [8]:
def nb_structure(text, item, tasks, links):
    
    text = text[text.find(tasks[links.index(item)]):]
    title = text[:text.find('\n')]
    definition = text[text.find('chance\n')+len('chance\n'):]
    results = definition[definition.find('\n\n'):]
    definition = definition[:definition.find('\n\n')]
    functions = results[results.find('def '):]
    functions = functions[:functions.find('Go')]

    results = results[results.find('Expected Run') + len('Expected Run')+1:]
    results = results[:results.rfind('OK')]

    topic = title[title.find(' > ') + len(' > '):]

    try:
        inputs = [eval(item[:item.find(')')]) for item in results.split(topic + '(')[1:]]
    except:
        inputs = ''
    try:    
        expected = [eval(item.replace(', ',',')[:item.replace(', ',',').find(' ')+1]) for item in results.split(' → ')[1:]]
    except:
        try:
            expected = [eval(item[:item.find('] [')+1]) for item in results.split(' → ')[1:]]
        except:
            expected = [item[1:item.find("' '")] for item in results.split(' → ')[1:]]
    
    parameters = functions[functions.find('('):functions.find(')')]

    if '],' in parameters:
        parameters = parameters.split('],')
        params = '('
        for i in range(len(parameters)):
            params += 'i[' + str(i) + ']'
            if i != len(parameters) - 1:
                params += ','
            else:
                params += ')'
        params += ')'
    elif ',' in parameters:
        parameters = parameters.split(',')
        params = '('
        for i in range(len(parameters)):
            params += 'i[' + str(i) + ']'
            if i != len(parameters) - 1:
                params += ','
            else:
                params += ')'
        params += ')'
    elif parameters == '(':
        params = '()'
    else:
        params = '(i))'

    body = "testers = " + json.dumps(inputs) + """

expected = """ + json.dumps(expected) + """

results =[]

for i in testers:
    results.append(""" + str(topic) + params + """

print("The function's returned result is equivalent to the expected result? " + str(results == expected))"""

    if params == '()':
        body = body.replace("""results =[]

for i in testers:
    results.append(""" + str(topic) + params, "results = [" + str(topic) + params + "]")

    text = """\
## """ + title + '\n' + definition
    
    return body.replace('true','True').replace('false','False'), text, functions, title

In [ ]:
page_link = "https://codingbat.com/python"

driver = opening_url(page_link)

credentials(driver)

tasks, links = topics_links(driver)

zoom_out()

for it in links:
    
    nb = nbf.v4.new_notebook()
    
    nb['cells'] = []

    prob = exercises_links(it)
    print(prob)

    for i in prob:

        text = get_content(i)
        
        pyautogui.scroll(-11)

        body, text, functions, title = nb_structure(text, it, tasks, links)

        img = printscreen(title.replace(' > ','_'))
        
        img = '<img src="' + img + '">'
        
        nb['cells'] += [nbf.v4.new_markdown_cell(text),
                       nbf.v4.new_code_cell(functions), nbf.v4.new_code_cell(body),
                       nbf.v4.new_markdown_cell("#### A screenshot of the condigbat's execise is shown below:"), nbf.v4.new_markdown_cell(img)]

        
    nbf.write(nb, tasks[links.index(it)] + '.ipynb')